### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [57]:
reviews

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,2,816711,8,1379963769,2013-09-23 19:16:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,2,1091191,7,1391173869,2014-01-31 13:11:09,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,2,1103275,7,1408192129,2014-08-16 12:28:49,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,2,1322269,7,1391529691,2014-02-04 16:01:31,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,2,1390411,8,1451374513,2015-12-29 07:35:13,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [59]:
top_movies = reviews[['rating', 'movie_id', 'date']].groupby('movie_id').agg(
        {'rating':['mean', 'count'], 'date':'max'})

In [40]:
top_movies

rating      
           mean count
movie_id             
4921860    10.0    48
5262972    10.0    28
5688932    10.0    14
2737018    10.0    10
2219210    10.0     6
2560840    10.0     6
1431149    10.0     5
2059318    10.0     5
4448444    10.0     5
5131914    10.0     5
71690      10.0     4
1230376    10.0     4
1472122    10.0     4
2254364    10.0     4
2455264    10.0     4
2621126    10.0     4
2788512    10.0     4
3303444    10.0     4
4107858    10.0     4
4730872    10.0     4
4942488    10.0     4
69847      10.0     3
79596      10.0     3
94739      10.0     3
109333     10.0     3
158446     10.0     3
245406     10.0     3
780524     10.0     3
865561     10.0     3
1703125    10.0     3
2004271    10.0     3
2328819    10.0     3
2375037    10.0     3
2415372    10.0     3
2473684    10.0     3
2543202    10.0     3
2915232    10.0     3
2952482    10.0     3
3181886    10.0     3
3381858    10.0     3
3427556    10.0     3
3429304    10.0     3
3443482    10.0     3
3640822    10.0     3
3739614    10.0     3
4772806    10.0     3
5918104    10.0     3
5974044    10.0     3
6076810    10.0     3
6467380    10.0     3

In [66]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    top_movies = reviews[['rating', 'movie_id', 'date']].groupby('movie_id').agg(
        {'rating':['mean', 'count'], 'date':'max'})
    top_movies.columns = ['mean', 'count', 'date']
    top_movies = top_movies.loc[top_movies['count']> 4]
    top_movies.sort_values(['mean', 'count', 'date'], ascending=False, inplace=True)
    top_movies = top_movies.join(movies[['movie_id','movie']].set_index('movie_id'))    
    
    return list(top_movies['movie'].head(n_top))

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [67]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35)



In [70]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [88]:
years = ['2015', '2016', '2017', '2018']
genres=['History']
# query = ' & '.join(["{} > 0".format(k) for k in years + genres])
mask = (x == 1) for x in years + genres

In [1]:

# query

In [ ]:
def popular_recs_filtered(user_id, n_top, years=[], genres=[]):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    top_movies = reviews[['rating', 'movie_id', 'date']].groupby('movie_id').agg(
        {'rating':['mean', 'count'], 'date':'max'})
    top_movies.columns = ['mean', 'count', 'date']
    top_movies = top_movies.loc[top_movies['count']> 4]
    top_movies.sort_values(['mean', 'count', 'date'], ascending=False, inplace=True)
    
    query = ' & '.join(['{} == 1'.format(k for k in years + genres)]
    
    filtered_movies = movies.loc[(movies['years']) & (movies['genre'].isin(genres))]
    top_movies = top_movies.join(movies[['movie_id','movie']].set_index('movie_id'))    
    
    return list(top_movies['movie'].head(n_top))